# Configuration

NOTES: The warnings after the import are referred to the fact that Tensorflow 2.x versions are built to directly look for a GPU in the system. The warning can be forgot if you are not going to use the GPU. 

In [1]:
!source myenv/bin/activate

In [2]:
LENGTH_CHOSEN = 126520

In [3]:
import os
import librosa
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import seaborn as sns
sns.set_style('whitegrid')
import IPython.display as ipd
import librosa.display
import numpy as np
import pickle
import scipy
import ipywidgets
import math

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.pipeline import make_pipeline
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, confusion_matrix
from scipy.cluster.hierarchy import dendrogram
from sklearn.cluster import AgglomerativeClustering

from tqdm import tqdm

import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, Conv2D, AveragePooling1D, MaxPooling2D, Flatten
from tensorflow.keras.optimizers import SGD, Adam 
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras import regularizers

# from livelossplot import PlotLossesKeras
tf.config.list_physical_devices('GPU')

2021-08-31 13:16:02.644167: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-08-31 13:16:02.644190: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-08-31 13:16:03.503797: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-08-31 13:16:03.504400: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-08-31 13:16:03.575767: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-08-31 13:16:03.575787: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (helemanc-Latitude-5410): /pr

[]

# Get Data from Dataset - normalized folder

In [4]:
TESS  = '/home/helemanc/Desktop/Binary_Model/normalized_audio/tess/' 


In [5]:
dir_list = os.listdir(TESS)
dir_list.sort()
dir_list

path = []
emotion = []
gender = []
actors = []

for fname in os.listdir(TESS):
    part_1 = fname.split('.')
    part_2 = part_1[0].split('_')
    i = part_2[0] + '_' + part_2[2]
    #for f in fname:
    if i == 'OAF_angry':
        emotion.append('angry')
        gender.append('female')
        actors.append('OAF')
    elif i == 'YAF_angry': 
        emotion.append('angry')
        gender.append('female')
        actors.append('YAF')


    elif i == 'OAF_disgust' :
        emotion.append('disgust')
        gender.append('female')
        actors.append('OAF')
    elif i == 'YAF_disgust': 
        emotion.append('disgust')
        gender.append('female')
        actors.append('YAF')


    elif i == 'OAF_fear':
        emotion.append('fear')
        gender.append('female')
        actors.append('OAF')
    elif i == 'YAF_fear': 
        emotion.append('fear')
        gender.append('female')
        actors.append('YAF') 


    elif i == 'OAF_happy' :
        emotion.append('happy')
        gender.append('female')
        actors.append('OAF')
    elif i == 'YAF_happy': 
        emotion.append('angry')
        gender.append('female')
        actors.append('YAF')            

    elif i == 'OAF_neutral':
        emotion.append('neutral')
        gender.append('female')
        actors.append('OAF')   
    elif i == 'YAF_neutral': 
        emotion.append('neutral')
        gender.append('female')
        actors.append('YAF')      


    elif i == 'OAF_ps':
        emotion.append('surprise')
        gender.append('female')
        actors.append('OAF')

    elif i == 'YAF_ps': 
        emotion.append('surprise')
        gender.append('female')
        actors.append('YAF')            

    elif i == 'OAF_sad':
        emotion.append('sadness')
        gender.append('female')
        actors.append('OAF')
    elif i == 'YAF_sad': 
        emotion.append('sadness')
        gender.append('female')
        actors.append('YAF')            
    else:
        emotion.append('Unknown')
    path.append(TESS + '/' + fname)


TESS_df = pd.DataFrame(emotion, columns = ['emotion_label'])
TESS_df = pd.concat([TESS_df, pd.DataFrame(gender, columns = ['gender']), 
                     pd.DataFrame(actors, columns= ['actors']),
                     pd.DataFrame(path, columns = ['path'])],axis=1)
TESS_df.emotion_label.value_counts()

angry       600
neutral     400
surprise    400
sadness     400
fear        400
disgust     400
happy       200
Name: emotion_label, dtype: int64

In [6]:
TESS_df= TESS_df[~TESS_df.path.str.contains('noise')]

In [7]:
TESS_train = []
TESS_test = []

In [8]:
for index, row in TESS_df.iterrows(): 
    if row['actors'] == 'YAF': 
        TESS_train.append(row)
    else: 
        TESS_test.append(row)
len(TESS_train), len(TESS_test)

(1400, 1400)

In [9]:
TESS_df.head()

,emotion_label,gender,actors,path
0,happy,female,OAF,/home/helemanc/Desktop/Binary_Model/normalized...
1,angry,female,YAF,/home/helemanc/Desktop/Binary_Model/normalized...
2,neutral,female,OAF,/home/helemanc/Desktop/Binary_Model/normalized...
3,surprise,female,OAF,/home/helemanc/Desktop/Binary_Model/normalized...
4,angry,female,OAF,/home/helemanc/Desktop/Binary_Model/normalized...


In [10]:
TESS_train = pd.DataFrame(TESS_train)
TESS_test = pd.DataFrame(TESS_test)

In [11]:
df_train = TESS_train

In [12]:
df_test = TESS_test

In [13]:
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

# Feature extraction

In [14]:
df_train['emotion_label'].unique()

array(['angry', 'sadness', 'fear', 'disgust', 'surprise', 'neutral'],
      dtype=object)

In [ ]:
plt.title('Emotions distribution')
plt.hist(df_train['emotion_label'])
# plt.hist(y)
plt.show()

In [ ]:
plt.title('Emotions distribution')
plt.hist(df_val['emotion_label'])
# plt.hist(y)
plt.show()

In [ ]:
plt.title('Emotions distribution')
plt.hist(df_test['emotion_label'])
# plt.hist(y)
plt.show()

In [ ]:
df_train.shape

## Utils

In [15]:
def load_files(df):
    X = []
    for i in tqdm(df['path']): 
        X.append(librosa.load(i, res_type='kaiser_fast', sr=16000))
    return X

def extract_samples(X): 
    samples = []
    for ind,i in enumerate(X):
        samples.append(i[0])
    return samples 

def extract_labels(df): 
    labels = df['emotion_label'].copy()
    return labels 

def compute_lengths(samples): 
    lengths = [len(x) for x in samples]
    return lengths 

def check_outliers(lengths):
    # outliers
    lengths = np.array(lengths)
    print((lengths > 300000).sum())
    new_lengths = lengths[lengths < 300000]
    return new_lengths 

def compute_mean_length(lengths): 
    return lengths.mean()

def cut_and_pad(samples, labels, length_chosen = LENGTH_CHOSEN): 
    X_new = []
    y_new = []
    count = 0 
    for ind,i in enumerate(samples):
        if i.shape[0] < 300000:
            if i.shape[0] > length_chosen:
                new = i[:length_chosen]
                X_new.append(new)
            elif i.shape[0] < length_chosen:
                new = np.pad(i,math.ceil((length_chosen-i.shape[0])/2), mode='median')
                X_new.append(new)
            else:
                X_new.append(i)
            y_new.append(labels[count])
        count+=1
    
    return X_new, y_new

# Data Augmentation 
def noise(data):
    noise_amp = 0.035*np.random.uniform()*np.amax(data)
    data = data + noise_amp*np.random.normal(size=data.shape[0])
    return data

# Data Augmentation 
def pitch(data, sampling_rate, pitch_factor=0.7):
    return librosa.effects.pitch_shift(data, sampling_rate, pitch_factor)
    
def compute_mfccs(samples): 
    mfccs = []
    for i in tqdm(samples):
        mfcc = librosa.feature.mfcc(y=i, sr=16000, n_mfcc=13)
        mfcc = mfcc.T
        mfcc = np.array(mfcc)
        mfccs.append(mfcc[:, 1:])
    mfccs = np.array(mfccs)
    return mfccs

def compute_mfccs_augmentation(samples, labels): 
    mfccs = []
    counter = 0 
    for i in tqdm(samples):

       # Weiner Filtering on original noise 
        samples_weiner = scipy.signal.wiener(i)
        is_fin = np.isfinite(samples_weiner).all()


        # Data Augmentation - Noise 
        noise_audio = noise(samples_weiner)

        # Data Augmentation - Pitch 
        pitch_audio = pitch(samples_weiner, sampling_rate=16000)


        # Data Augmentation -  pitch + noise 
        pn = pitch(noise_audio, sampling_rate = 16000)


        if is_fin: 
          # MFCC

          mfcc = librosa.feature.mfcc(y=i, sr=16000, n_mfcc=13)
          mfcc = mfcc.T
          mfccs.append(mfcc[:, 1:])

          mfcc_augmented = librosa.feature.mfcc(y=samples_weiner, sr=16000, n_mfcc=13)
          mfcc_augmented = mfcc_augmented.T
          mfccs.append(mfcc_augmented[:, 1:])

          mfcc_augmented_pitch = librosa.feature.mfcc(y=noise_audio, sr=16000, n_mfcc=13)
          mfcc_augmented_pitch = mfcc_augmented_pitch.T
          mfccs.append(mfcc_augmented_pitch[:, 1:])

          mfcc_augmented_p = librosa.feature.mfcc(y=pitch_audio, sr=16000, n_mfcc=13)
          mfcc_augmented_p = mfcc_augmented_p.T
          mfccs.append(mfcc_augmented_p[:, 1:]) 

          mfcc_augmented_pn = librosa.feature.mfcc(y=pn, sr=16000, n_mfcc=13)
          mfcc_augmented_pn = mfcc_augmented_pn.T
          mfccs.append(mfcc_augmented_pn[:, 1:]) 
    
    mfccs = np.array(mfccs)
    
    # Copy labels 
    y_prov = []
    y = labels 
    for i in range(len(y)): 
      y_prov.append(y[i])
      y_prov.append(y[i])
      y_prov.append(y[i])
      y_prov.append(y[i])
      y_prov.append(y[i])
    y = np.asarray(y_prov)

    return mfccs, y 





## Train

### Load samples and labels 

In [16]:
load_train = load_files(df_train)

100%|█████████████████████████████████████| 1400/1400 [00:00<00:00, 4669.38it/s]


In [17]:
samples_train = extract_samples(load_train)

In [18]:
labels_train = extract_labels(df_train)

In [19]:
labels_train

0          angry
1        sadness
2           fear
3        disgust
4       surprise
          ...   
1395       angry
1396     disgust
1397       angry
1398     disgust
1399        fear
Name: emotion_label, Length: 1400, dtype: object

### Decide length

In [ ]:
lengths = compute_lengths(samples_train) 

In [ ]:
new_lengths = check_outliers(lengths) 

In [ ]:
mean_length = compute_mean_length(new_lengths)

In [ ]:
mean_length

### Cut and Pad

In [20]:
samples_train, labels_train = cut_and_pad(samples_train, labels_train)

In [21]:
samples_train = np.array(samples_train)
labels_train = np.array(labels_train)
print(samples_train.shape, labels_train.shape)

(1400,) (1400,)


### Feature Extraction - Without Data Augmentation 

In [22]:
mfccs_train = compute_mfccs(samples_train)

100%|███████████████████████████████████████| 1400/1400 [00:23<00:00, 60.61it/s]


In [23]:
mfccs_train.shape

(1400, 248, 12)

### Save features and labels

In [ ]:
'''
preprocess_path = "/home/helemanc/Desktop/Binary_Model/pre-processed"
mfccs_train_path = os.path.join(preprocess_path,"mfccs_train.pkl")
labels_train_path = os.path.join(preprocess_path,"labels_train.pkl")

with open(mfccs_train_path, 'wb') as f:
        pickle.dump(mfccs_train , f)
        
with open(labels_train_path, 'wb') as f:
        pickle.dump(labels_train , f)
'''

## Test

### Load samples and labels

In [24]:
load_test = load_files(df_test)

100%|█████████████████████████████████████| 1400/1400 [00:00<00:00, 4009.09it/s]


In [25]:
samples_test = extract_samples(load_test)

In [26]:
labels_test = extract_labels(df_test)

### Cut and Pad 

In [27]:
samples_test, labels_test = cut_and_pad(samples_test, labels_test)

In [28]:
samples_test = np.array(samples_test)
labels_test = np.array(labels_test)
print(samples_test.shape, labels_test.shape)

(1400,) (1400,)


### Feature Extraction

In [29]:
mfccs_test = compute_mfccs(samples_test)

100%|███████████████████████████████████████| 1400/1400 [00:24<00:00, 58.26it/s]


In [30]:
mfccs_test.shape

(1400, 248, 12)

# Encode Labels - Binary 

In [31]:
emotion_enc = {'fear':1, 'disgust':1, 'neutral':0, 'calm':0,  'happy':0, 'sadness':1, 'surprise':0, 'angry':1}

In [32]:
y_train = pd.Series(labels_train).replace(emotion_enc)
#y_val = pd.Series(labels_val).map(emotion_enc)
y_test = pd.Series(labels_test).map(emotion_enc)
#y_train_aug = pd.Series(labels_train_aug).map(emotion_enc)

# Train, Val, Test (X) 

In [33]:
X_train = mfccs_train
#X_val = mfccs_val
X_test = mfccs_test 

# Standard Scaling 

## Without Augmentation

In [34]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
X_test = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)
#X_val = scaler.transform(X_val.reshape(-1, X_val.shape[-1])).reshape(X_val.shape)

# Model 

In [ ]:
model = Sequential()

model.add(layers.Conv1D(256, 5,padding='same',
                 input_shape=(248,12), kernel_initializer=tf.keras.initializers.HeNormal(seed=0)))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling1D(pool_size=(4)))
model.add(layers.Dropout(0.6))

model.add(layers.Conv1D(128, 5,padding='same', kernel_initializer=tf.keras.initializers.HeNormal(seed=0)))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling1D(pool_size=(4)))
model.add(layers.Dropout(0.5))

model.add(layers.Flatten())
model.add(layers.Dense(64,  kernel_initializer=tf.keras.initializers.HeNormal(seed=0)))
model.add(layers.Dense(1))
model.add(layers.Activation('sigmoid'))

model.summary()

# Training

## Without Augmentation

### Load Data

In [ ]:
'''
X_train = pickle.load(open( "/home/helemanc/Desktop/Binary_Model/data_model/X_train.pkl", 'rb'))
X_val = pickle.load(open( "/home/helemanc/Desktop/Binary_Model/data_model/X_val.pkl", 'rb'))
X_test = pickle.load(open( "/home/helemanc/Desktop/Binary_Model/data_model/X_test.pkl", 'rb'))
y_train = pickle.load(open( "/home/helemanc/Desktop/Binary_Model/data_model/y_train.pkl", 'rb'))
y_val = pickle.load(open( "/home/helemanc/Desktop/Binary_Model/data_model/y_val.pkl", 'rb'))
y_test =  pickle.load(open( "/home/helemanc/Desktop/Binary_Model/data_model/y_test.pkl", 'rb'))
'''

In [ ]:
from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train, y_train)

In [ ]:
weight_path = "/home/helemanc/Desktop/Binary_Model/weights/binary_model_l1l2.hdf5"

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', 
                                                 factor=0.5, patience=4, 
                                                 verbose=1, mode='max', 
                                                 min_lr=0.00001)

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=45, 
                                              verbose=1)

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=weight_path, 
                                                      save_weights_only=True, 
                                                      monitor='val_accuracy', 
                                                      mode='max', 
                                                      save_best_only=True)
# classweight 
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)
class_weights = {l:c for l,c in zip(np.unique(y_train), class_weights)}


In [ ]:
model.compile(loss='binary_crossentropy', optimizer="adam", metrics='accuracy')

### Train

In [ ]:
history = model.fit(X_train, y_train, batch_size=4, epochs=500, validation_data=(X_val, y_val),
           callbacks=[reduce_lr, early_stop, model_checkpoint], class_weight = class_weights)

### Plot Training Graphs

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('acc')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

### Save Model

In [ ]:
model.save("/home/helemanc/Desktop/Binary_Model/models/binary_model_ravdess_13_83")

### Evaluate Model

In [ ]:
model_loaded = tf.keras.models.load_model("/home/helemanc/Desktop/Binary_Model/models/binary_model")

In [ ]:
model.evaluate(X_test, y_test, batch_size=8)

In [ ]:
from sklearn.metrics import classification_report
predictions = model.predict(X_test)
pred = [1 * (x[0]>=0.50) for x in predictions] #0.5 o 0.52? 
print(classification_report(y_test, pred))

## With Augmentation

### Load Data

In [ ]:
'''
X_train_aug = pickle.load(open( "/home/helemanc/Desktop/Binary_Model/data_model_augmented/X_train_aug.pkl", 'rb'))
X_val = pickle.load(open( "/home/helemanc/Desktop/Binary_Model/data_model_augmented/X_val.pkl", 'rb'))
X_test = pickle.load(open( "/home/helemanc/Desktop/Binary_Model/data_model_augmented/X_test.pkl", 'rb'))
y_train_aug = pickle.load(open( "/home/helemanc/Desktop/Binary_Model/data_model_augmented/y_train_aug.pkl", 'rb'))
y_val = pickle.load(open( "/home/helemanc/Desktop/Binary_Model/data_model_augmented/y_val.pkl", 'rb'))
y_test =  pickle.load(open( "/home/helemanc/Desktop/Binary_Model/data_model_augmented/y_test.pkl", 'rb'))
'''

In [ ]:
X_train_aug = mfccs_train_aug
y_train_aug = labels_train_aug 

In [ ]:
from sklearn.utils import shuffle
X_train_aug, y_train_aug = shuffle(X_train_aug, y_train_aug)

In [ ]:
(unique, counts) = np.unique(y_train_aug, return_counts=True)
frequencies = np.asarray((unique, counts)).T
print(frequencies)

In [ ]:
weight_path = "/home/helemanc/Desktop/Binary_Model/weights/binary_model_augmented.hdf5"

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', 
                                                 factor=0.5, patience=4, 
                                                 verbose=1, mode='max', 
                                                 min_lr=0.00001)

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=45, 
                                              verbose=1)

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=weight_path, 
                                                      save_weights_only=True, 
                                                      monitor='val_accuracy', 
                                                      mode='max', 
                                                      save_best_only=True)
# classweight 
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train_aug), y_train_aug)
class_weights = {l:c for l,c in zip(np.unique(y_train_aug), class_weights)}


In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics='accuracy')

### Train

In [ ]:
history = model.fit(X_train_aug, y_train_aug, batch_size=16, epochs=500, validation_data=(X_val, y_val),
           callbacks=[reduce_lr, early_stop, model_checkpoint], class_weight = class_weights)


### Plot Training Graphs

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('acc')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

### Save Model

In [ ]:
model.save("/home/helemanc/Desktop/Binary_Model/models/binary_model_aug")

### Evaluate Model

In [ ]:
model_loaded = tf.keras.models.load_model("/home/helemanc/Desktop/Binary_Model/models/binary_model_aug")

In [ ]:
model_loaded.evaluate(X_test, y_test, batch_size=16)

In [ ]:
from sklearn.metrics import classification_report
predictions = model_loaded.predict(X_test)
pred = [1 * (x[0]>=0.50) for x in predictions] #0.5 o 0.52? 
print(classification_report(y_test, pred))

# Hyprparameter optimization 

In [ ]:
def create_model(optimizer='adam', init_mode='glorot_uniform'):
    model = Sequential()

    model.add(layers.Conv1D(256, 5,padding='same',
                     input_shape=(248,12), kernel_initializer=tf.keras.initializers.HeNormal(seed=0)))
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling1D(pool_size=(4)))
    model.add(layers.Dropout(0.6))

    model.add(layers.Conv1D(128, 5,padding='same', kernel_initializer=tf.keras.initializers.HeNormal(seed=0)))
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling1D(pool_size=(4)))
    model.add(layers.Dropout(0.5))

    model.add(layers.Flatten())
    model.add(layers.Dense(64,  kernel_initializer=tf.keras.initializers.HeNormal(seed=0)))
    model.add(layers.Dense(1))
    model.add(layers.Activation('sigmoid'))
    
    # compile model
    model.compile(loss='binary_crossentropy', 
                  optimizer=optimizer, 
                  metrics=['accuracy'])
    return model

In [ ]:
from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train, y_train)

## Initializer

In [ ]:
weight_path = "/home/helemanc/Desktop/Binary_Model/weights/binary_model_l1l2.hdf5"

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='accuracy', 
                                                 factor=0.5, patience=4, 
                                                 verbose=1, mode='max', 
                                                 min_lr=0.00001)

early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=45, 
                                              verbose=1)

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=weight_path, 
                                                      save_weights_only=True, 
                                                      monitor='val_accuracy', 
                                                      mode='max', 
                                                      save_best_only=True)
# classweight 
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)
class_weights = {l:c for l,c in zip(np.unique(y_train), class_weights)}


In [ ]:
%%time
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import KFold
#import tensorflow as tf 
#from tf.keras.wrappers.scikit_learn import KerasClassifier

seed = 7
np.random.seed(seed)
batch_size = 4
epochs = 30

model_CV = tf.keras.wrappers.scikit_learn.KerasClassifier(build_fn=create_model, epochs=epochs, 
                           batch_size=batch_size, verbose=2)
# define the grid search parameters
init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 
             'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']

param_grid = dict(init_mode=init_mode)
grid = RandomizedSearchCV(estimator=model_CV, param_distributions=param_grid, n_jobs=-1, cv=KFold(3), verbose = 10 )
grid_result = grid.fit(X_train, y_train, callbacks=[reduce_lr, early_stop], class_weight = class_weights)

In [ ]:
# print results
print(f'Best Accuracy for {grid_result.best_score_} using {grid_result.best_params_}')
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f' mean={mean:.4}, std={stdev:.4} using {param}')

## Initializer, Batch Size, Learning Rate 

In [35]:
def create_model( init_mode='glorot_uniform', lr = 0.001):
    model = Sequential()

    model.add(layers.Conv1D(256, 5,padding='same',
                     input_shape=(248,12), kernel_initializer=init_mode))
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling1D(pool_size=(4)))
    model.add(layers.Dropout(0.6))

    model.add(layers.Conv1D(128, 5,padding='same', kernel_initializer=init_mode))
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling1D(pool_size=(4)))
    model.add(layers.Dropout(0.5))

    model.add(layers.Flatten())
    model.add(layers.Dense(64, kernel_initializer=init_mode))
    model.add(layers.Dense(1))
    model.add(layers.Activation('sigmoid'))
    
    # compile model
    model.compile(loss='binary_crossentropy', 
                  optimizer=Adam(lr = lr) , 
                  metrics=['accuracy'])
    return model

In [36]:
weight_path = "/home/helemanc/Desktop/Binary_Model/weights/binary_model_l1l2.hdf5"

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='accuracy', 
                                                 factor=0.5, patience=4, 
                                                 verbose=1, mode='max', 
                                                 min_lr=0.000001)

early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=45, 
                                              verbose=1)

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=weight_path, 
                                                      save_weights_only=True, 
                                                      monitor='val_accuracy', 
                                                      mode='max', 
                                                      save_best_only=True)
# classweight 
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)
class_weights = {l:c for l,c in zip(np.unique(y_train), class_weights)}


In [37]:
%%time
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import KFold, StratifiedKFold
#import tensorflow as tf 
#from tf.keras.wrappers.scikit_learn import KerasClassifier

seed = 7
np.random.seed(seed)
batch_size = 4
epochs = 50

model_CV = tf.keras.wrappers.scikit_learn.KerasClassifier(build_fn=create_model, epochs=epochs, 
                           batch_size=batch_size, verbose=2)
# define the grid search parameters
init_mode = ['uniform', 'lecun_uniform', 'glorot_uniform', 'glorot_normal']
batches = [4,8,16]
lr = [0.001, 0.0001, 0.00005]

param_grid = dict(init_mode=init_mode, lr = lr, batch_size = batches)
grid = RandomizedSearchCV(estimator=model_CV, param_distributions=param_grid, n_jobs=-1, cv=StratifiedKFold(3))
grid_result = grid.fit(X_train, y_train, callbacks=[reduce_lr, early_stop], class_weight = class_weights)

2021-08-31 13:17:22.856549: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-08-31 13:17:22.856594: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-08-31 13:17:22.857036: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-08-31 13:17:22.857053: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-08-31 13:17:22.857425: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or dire

Epoch 1/50
59/59 - 7s - loss: 0.4080 - accuracy: 0.8156
Epoch 1/50
59/59 - 7s - loss: 0.7605 - accuracy: 0.5857
Epoch 1/50
59/59 - 7s - loss: 0.7292 - accuracy: 0.5949
Epoch 1/50
59/59 - 8s - loss: 0.8469 - accuracy: 0.5316
Epoch 1/50
59/59 - 8s - loss: 0.3278 - accuracy: 0.8371
Epoch 1/50
234/234 - 8s - loss: 0.6376 - accuracy: 0.6259
Epoch 1/50
234/234 - 8s - loss: 0.6084 - accuracy: 0.6531
Epoch 1/50
234/234 - 9s - loss: 0.6236 - accuracy: 0.6484
Epoch 2/50
59/59 - 7s - loss: 0.0712 - accuracy: 0.9796
Epoch 2/50
59/59 - 7s - loss: 0.6037 - accuracy: 0.6785
Epoch 2/50
59/59 - 7s - loss: 0.5648 - accuracy: 0.7010
Epoch 2/50
59/59 - 7s - loss: 0.6189 - accuracy: 0.6649
Epoch 2/50
59/59 - 7s - loss: 0.0660 - accuracy: 0.9743
Epoch 2/50
234/234 - 8s - loss: 0.4027 - accuracy: 0.8296
Epoch 2/50
234/234 - 8s - loss: 0.4284 - accuracy: 0.8210
Epoch 2/50
234/234 - 8s - loss: 0.4051 - accuracy: 0.8458
Epoch 3/50
59/59 - 7s - loss: 0.0305 - accuracy: 0.9893
Epoch 3/50
59/59 - 7s - loss: 0.5393

2021-08-31 13:39:44.807664: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-08-31 13:39:44.808484: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-08-31 13:39:44.969329: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-08-31 13:39:44.991703: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2299965000 Hz


Epoch 1/50
350/350 - 2s - loss: 0.5687 - accuracy: 0.6843
Epoch 2/50
350/350 - 1s - loss: 0.2699 - accuracy: 0.9093
Epoch 3/50
350/350 - 1s - loss: 0.1475 - accuracy: 0.9579
Epoch 4/50
350/350 - 1s - loss: 0.0950 - accuracy: 0.9736
Epoch 5/50
350/350 - 1s - loss: 0.0584 - accuracy: 0.9871
Epoch 6/50
350/350 - 1s - loss: 0.0462 - accuracy: 0.9871
Epoch 7/50
350/350 - 1s - loss: 0.0344 - accuracy: 0.9900
Epoch 8/50
350/350 - 1s - loss: 0.0283 - accuracy: 0.9936
Epoch 9/50
350/350 - 1s - loss: 0.0312 - accuracy: 0.9893
Epoch 10/50
350/350 - 1s - loss: 0.0210 - accuracy: 0.9921
Epoch 11/50
350/350 - 1s - loss: 0.0162 - accuracy: 0.9950
Epoch 12/50
350/350 - 1s - loss: 0.0150 - accuracy: 0.9964
Epoch 13/50
350/350 - 1s - loss: 0.0140 - accuracy: 0.9964
Epoch 14/50
350/350 - 1s - loss: 0.0092 - accuracy: 0.9979
Epoch 15/50
350/350 - 1s - loss: 0.0097 - accuracy: 0.9964
Epoch 16/50
350/350 - 1s - loss: 0.0088 - accuracy: 0.9979
Epoch 17/50
350/350 - 1s - loss: 0.0140 - accuracy: 0.9964
Epoch 

In [38]:
# print results
print(f'Best Accuracy for {grid_result.best_score_} using {grid_result.best_params_}')
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f' mean={mean:.4}, std={stdev:.4} using {param}')

Best Accuracy for 1.0 using {'lr': 5e-05, 'init_mode': 'uniform', 'batch_size': 4}
 mean=0.9993, std=0.001009 using {'lr': 5e-05, 'init_mode': 'glorot_uniform', 'batch_size': 16}
 mean=1.0, std=0.0 using {'lr': 5e-05, 'init_mode': 'uniform', 'batch_size': 4}
 mean=1.0, std=0.0 using {'lr': 0.001, 'init_mode': 'uniform', 'batch_size': 16}
 mean=1.0, std=0.0 using {'lr': 0.0001, 'init_mode': 'uniform', 'batch_size': 4}
 mean=1.0, std=0.0 using {'lr': 0.001, 'init_mode': 'glorot_uniform', 'batch_size': 8}
 mean=0.9993, std=0.001009 using {'lr': 0.0001, 'init_mode': 'uniform', 'batch_size': 8}
 mean=1.0, std=0.0 using {'lr': 0.001, 'init_mode': 'glorot_normal', 'batch_size': 4}
 mean=1.0, std=0.0 using {'lr': 0.001, 'init_mode': 'lecun_uniform', 'batch_size': 8}
 mean=1.0, std=0.0 using {'lr': 0.0001, 'init_mode': 'glorot_normal', 'batch_size': 8}
 mean=0.9986, std=0.002023 using {'lr': 5e-05, 'init_mode': 'lecun_uniform', 'batch_size': 4}


## Train with best parameters

In [39]:
#Best Accuracy for 0.6906271775563558 using {'lr': 0.001, 'init_mode': 'lecun_uniform', 'batch_size': 8}
# Best Accuracy for 1.0 using {'lr': 5e-05, 'init_mode': 'uniform', 'batch_size': 4}
def create_model( init_mode='uniform', lr = 5e-05):
    model = Sequential()

    model.add(layers.Conv1D(256, 5,padding='same',
                     input_shape=(248,12), kernel_initializer=init_mode))
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling1D(pool_size=(4)))
    model.add(layers.Dropout(0.6))

    model.add(layers.Conv1D(128, 5,padding='same', kernel_initializer=init_mode))
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling1D(pool_size=(4)))
    model.add(layers.Dropout(0.5))

    model.add(layers.Flatten())
    model.add(layers.Dense(64, kernel_initializer=init_mode))
    model.add(layers.Dense(1))
    model.add(layers.Activation('sigmoid'))
    
    # compile model
    model.compile(loss='binary_crossentropy', 
                  optimizer=Adam(lr = lr) , 
                  metrics=['accuracy'])
    return model

In [40]:
seed = 7
np.random.seed(seed)

In [41]:
model = create_model()

In [42]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [43]:
import datetime, os

In [44]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

In [45]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

2021-08-31 14:48:59.181427: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2021-08-31 14:48:59.181458: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2021-08-31 14:48:59.233834: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.


In [46]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='accuracy', 
                                                 factor=0.5, patience=4, 
                                                 verbose=1, mode='max', 
                                                 min_lr=0.000001)

early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=45, 
                                              verbose=1, restore_best_weights = True )

# classweight 
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)
class_weights = {l:c for l,c in zip(np.unique(y_train), class_weights)}

In [47]:
history = model.fit(X_train, y_train, batch_size=4, epochs=500,
           callbacks=[reduce_lr, early_stop, tensorboard_callback], class_weight = class_weights)

Epoch 1/500
 24/350 [=>............................] - ETA: 2s - loss: 1.0393 - accuracy: 0.5074

2021-08-31 14:49:09.530814: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2021-08-31 14:49:09.530837: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2021-08-31 14:49:09.539184: I tensorflow/core/profiler/lib/profiler_session.cc:71] Profiler session collecting data.
2021-08-31 14:49:09.546790: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.
2021-08-31 14:49:09.556124: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: logs/20210831-144856/train/plugins/profile/2021_08_31_14_49_09
2021-08-31 14:49:09.558822: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for trace.json.gz to logs/20210831-144856/train/plugins/profile/2021_08_31_14_49_09/helemanc-Latitude-5410.trace.json.gz
2021-08-31 14:49:09.572331: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: logs/20210831-144856/train/plugins/p

350/350 [==============================] - 2s 5ms/step - loss: 0.7104 - accuracy: 0.5638
Epoch 2/500
350/350 [==============================] - 2s 5ms/step - loss: 0.3404 - accuracy: 0.8804
Epoch 3/500
350/350 [==============================] - 2s 5ms/step - loss: 0.1823 - accuracy: 0.9476
Epoch 4/500
350/350 [==============================] - 2s 5ms/step - loss: 0.1119 - accuracy: 0.9770
Epoch 5/500
350/350 [==============================] - 2s 5ms/step - loss: 0.0655 - accuracy: 0.9829
Epoch 6/500
350/350 [==============================] - 1s 4ms/step - loss: 0.0585 - accuracy: 0.9888
Epoch 7/500
350/350 [==============================] - 1s 4ms/step - loss: 0.0346 - accuracy: 0.9945
Epoch 8/500
350/350 [==============================] - 1s 4ms/step - loss: 0.0377 - accuracy: 0.9892
Epoch 9/500
350/350 [==============================] - 1s 4ms/step - loss: 0.0306 - accuracy: 0.9929
Epoch 10/500
350/350 [==============================] - 1s 4ms/step - loss: 0.0206 - accuracy: 0.9962
E

In [48]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 31441), started 1 day, 3:29:02 ago. (Use '!kill 31441' to kill it.)

In [49]:
model.evaluate(X_test, y_test, batch_size=4)

350/350 [==============================] - 1s 1ms/step - loss: 3.4395 - accuracy: 0.5336


[3.4394803047180176, 0.5335714221000671]

In [50]:
from sklearn.metrics import classification_report
predictions = model.predict(X_test)
pred = [1 * (x[0]>=0.50) for x in predictions] #0.5 o 0.52? 
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.25      0.04      0.08       600
           1       0.56      0.90      0.69       800

    accuracy                           0.53      1400
   macro avg       0.40      0.47      0.38      1400
weighted avg       0.43      0.53      0.43      1400

